# Assignment V: GitHub and the ticketmaster.com API

## General

GitHub repository: https://github.com/Borschu/Assignment_05

Import packages:

In [18]:
import numpy as np
import pandas as pd
import requests
import time

## Setting up a new GitHub repository

see link above

## Getting to know the API

### (5)

Rate limits:
- 5000 API calls per day
- 5 requests per second
- you can only retrieve the 1000 th item (size * page < 1000)


### (6)

In [22]:
# create connection to API key
#with open(r"C:\Users\schur\OneDrive\Dokumente\Studium\Data Science Master Tübingen\1. Semester\Data Science Project Management\Assignment\key_assignment_5_python.py") as script:
#    exec(script.readline())

with open("C:/Users/schur/OneDrive/Dokumente/Studium/Data Science Master Tübingen/1. Semester/Data Science Project Management/Assignment/key_assignment_5_python.py") as script:
  exec(script.readline())


## Interacting with the API - the basics

### (7)

In [21]:
# create content for venue = Germany
con_ven_ger = requests.get("https://app.ticketmaster.com/discovery/v2/venues",
                          params = {"apikey": ticketmaster_key, "countryCode": "DE"}).json()

# print content
print(con_ven_ger)

{'_embedded': {'venues': [{'name': 'Gruenspan', 'type': 'venue', 'id': 'KovZpZAneakA', 'test': False, 'url': 'http://www.ticketmaster.de/venue/287155', 'locale': 'en-de', 'images': [{'ratio': '16_9', 'url': 'https://s1.ticketm.net/dbimages/2057v.', 'width': 205, 'height': 115, 'fallback': False}], 'postalCode': '22767', 'timezone': 'Europe/Berlin', 'city': {'name': 'Hamburg'}, 'country': {'name': 'Germany', 'countryCode': 'DE'}, 'address': {'line1': 'Grosse Freiheit 58'}, 'location': {'longitude': '9.958075', 'latitude': '53.551885'}, 'markets': [{'name': 'Germany', 'id': '210'}], 'dmas': [{'id': 610}], 'boxOfficeInfo': {'phoneNumberDetail': 'Gruenspan Große Freiheit 58 22767 Hamburg Tel: 040-313616 mail: info@gruenspan.de web: www.gruenspan.de'}, 'upcomingEvents': {'_total': 2, 'mfx-de': 2, '_filtered': 0}, 'ada': {'adaPhones': '+49.(0)1805 - 969 0000 (14 Ct./Min.)', 'adaCustomCopy': 'Sollten Sie besondere Sitzplatzwünsche haben, prüfen Sie bitte die Informationsseite des Veranstaltun

### (8)

In [71]:
# define size of page
size = con_ven_ger['page']['size']

# create empty vectors for all characteristics
name = [None]*size
city = [None]*size
postalCode = [None]*size
address = [None]*size
url = [None]*size
longitude = [None]*size
latitude = [None]*size

# for loop over all list entries for every characteristic of the venue
for i in range(size):
    # If a list element doesnt exist a key error occurs and the loop stops.
    # To avoid this the try-except-structure is built
    # if a KeyError occurs the value is codes as NaN
    try:
        name[i] = con_ven_ger['_embedded']['venues'][i]["name"]
    except KeyError:
        name[i] = np.nan
    try:
        city[i] = con_ven_ger['_embedded']['venues'][i]["city"]['name']
    except KeyError:
        city[i] = np.nan
    try:
        postalCode[i] = con_ven_ger['_embedded']['venues'][i]["postalCode"]
    except KeyError:
        postalCode[i] = np.nan
    try:
        address[i] = con_ven_ger['_embedded']['venues'][i]["address"]['line1']
    except KeyError:
        address[i] = np.nan
    try:
        url[i] = con_ven_ger['_embedded']['venues'][i]["url"]
    except KeyError:
        url[i] = np.nan
    try:
        longitude[i] = con_ven_ger['_embedded']['venues'][i]["location"]["longitude"]
    except KeyError:
        longitude[i] = np.nan
    try:
        latitude[i] = con_ven_ger['_embedded']['venues'][i]["location"]["latitude"]
    except KeyError:
        latitude[i] = np.nan
    
# create dataframe based on the previous created lists
df_venue_ger = pd.DataFrame({"name": name,
                           "city": city,
                           "postalCode": postalCode,
                           "address": address,
                           "url": url,
                           "longitude": longitude,
                           "latitude": latitude})

# print dataframe
print(df_venue_ger)


                             name             city postalCode  \
0                       Gruenspan          Hamburg      22767   
1               Huxleys Neue Welt           Berlin      10967   
2                   Virtual Event        Worldwide        NaN   
3           Ev. St. Jacobi Kirche     Sangerhausen      06526   
4             Evangelische Kirche           Senden      48308   
5                       HDI Arena         Hannover        NaN   
6                         Arsenal           Berlin      10785   
7       Freilichtbühne Heppenheim       Heppenheim      64646   
8                Schlosswallhalle        Osnabrück      49074   
9                Metropol Theater           Vechta      49377   
10        RÖDERMARK - Kulturhalle        Rödermark      63322   
11                           3922          Tambach      96479   
12                Nibelungenhalle         Walldürn      74731   
13            St. Johannis Kirche         Würzburg      97070   
14                   film

## Interacting with the API - advanced

### (9)

In [82]:
def create_df(countryCode):
    # create content 
    con = requests.get("https://app.ticketmaster.com/discovery/v2/venues",
                          params = {"apikey": ticketmaster_key, "countryCode": countryCode}).json()
    
    time.sleep(0.2)
    
    size = con['page']['size']
    nr_pages = con['page']['totalPages']
    
    name = [None]*1
    city = [None]*1
    postalCode = [None]*1
    address = [None]*1
    url = [None]*1
    longitude = [None]*1
    latitude = [None]*1
    
    df_all = pd.DataFrame({"name": name,
                                   "city": city,
                                   "postalCode": postalCode,
                                   "address": address,
                                   "url": url,
                                   "longitude": longitude,
                                   "latitude": latitude})
    
    for p in range(nr_pages):
        con = requests.get("https://app.ticketmaster.com/discovery/v2/venues",
                          params = {"apikey": ticketmaster_key, "countryCode": countryCode, "page": p}).json()
        
        if p == nr_pages-1:
            size = (con['page']['totalElements'] % size)-1
        
        # create empty vectors for all characteristics
        name = [None]*size
        city = [None]*size
        postalCode = [None]*size
        address = [None]*size
        url = [None]*size
        longitude = [None]*size
        latitude = [None]*size

        # for loop over all list entries for every characteristic of the venue
        for i in range(size):
            # If a list element doesnt exist a key error occurs and the loop stops.
            # To avoid this the try-except-structure is built
            # if a KeyError occurs the value is codes as NaN
            try:
                name[i] = con['_embedded']['venues'][i]["name"]
            except KeyError:
                name[i] = np.nan
            try:
                city[i] = con['_embedded']['venues'][i]["city"]['name']
            except KeyError:
                city[i] = np.nan
            try:
                postalCode[i] = con['_embedded']['venues'][i]["postalCode"]
            except KeyError:
                postalCode[i] = np.nan
            try:
                address[i] = con['_embedded']['venues'][i]["address"]['line1']
            except KeyError:
                address[i] = np.nan
            try:
                url[i] = con['_embedded']['venues'][i]["url"]
            except KeyError:
                url[i] = np.nan
            try:
                longitude[i] = con['_embedded']['venues'][i]["location"]["longitude"]
            except KeyError:
                longitude[i] = np.nan
            try:
                latitude[i] = con['_embedded']['venues'][i]["location"]["latitude"]
            except KeyError:
                latitude[i] = np.nan

        # create dataframe based on the previous created lists
        df_venue = pd.DataFrame({"name": name,
                                   "city": city,
                                   "postalCode": postalCode,
                                   "address": address,
                                   "url": url,
                                   "longitude": longitude,
                                   "latitude": latitude})
        df_all = pd.concat([df_all,df_venue])
        print(p)
        time.sleep(0.2)
        
    return df_all

# create Dataframe for Germany
df_ger = create_df("DE")

# print first 10 rows of Dataframe
print(df_ger.head(10))

# print shape of dataframe
print(df_ger.shape())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237


,name,city,postalCode,address,url,longitude,latitude
0,None,None,None,None,None,None,None
0,Gruenspan,Hamburg,22767,Grosse Freiheit 58,http://www.ticketmaster.de/venue/287155,9.958075,53.551885
1,Huxleys Neue Welt,Berlin,10967,Hasenheide 107 – 113,http://www.ticketmaster.de/venue/286842,13.42138,52.486391
2,Virtual Event,Worldwide,NaN,NaN,https://www.ticketmaster.de/venue/virtuelles-e...,10.0,50.0
3,Ev. St. Jacobi Kirche,Sangerhausen,06526,Marktplatz,http://www.ticketmaster.de/venue/290061,NaN,NaN
...,...,...,...,...,...,...,...
19,Rudolf Harbig Stadion,Dresden,01067,Lennestrabe 12,NaN,0,0
0,PreZero Arena,Sinsheim,74889,Baden-Wurttemberg,NaN,0,0
1,BayArena,Leverkusen,51371,Bismarckstrake 122,NaN,0,0
2,Impuls Arena,Augsburg,86150,Burgermeister Ulrich-strasse 90,NaN,0,0


In [ ]:
##